In [1]:
# In order to always have the same set for training and for evaluation I do the split here and
# then load it into each model

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import pandas as pd
import anndata

In [16]:
drugs = pd.read_csv('/lustre/groups/ml01/projects/CellOT_comparison/cellflow/data/4i/drugs.txt', header=None)[0].values
drugs = [a[:-1] if a!='vindesine' else a for a in drugs]
adata = sc.read_h5ad('/lustre/groups/ml01/projects/CellOT_comparison/cellflow/data/4i/8h.h5ad')
adata

AnnData object with n_obs × n_vars = 119479 × 78
    obs: 'drug'
    var: 'cell_part', 'measurement', 'stat', 'feature', 'original'
    uns: 'source'

In [17]:
adata = adata[adata.obs.drug.isin([*drugs, 'control'])].copy()
adata

AnnData object with n_obs × n_vars = 97748 × 78
    obs: 'drug'
    var: 'cell_part', 'measurement', 'stat', 'feature', 'original'
    uns: 'source'

In [18]:
frac = 0.4

adata0 = adata[adata.obs.drug=='control'].copy()
df_control_test = adata0.obs.sample(frac=frac)

for d in drugs:
    df_drug_test = adata[adata.obs.drug==d].obs.sample(frac=frac)
    
    dfc = adata[adata.obs.drug.isin(['control', d])].obs.copy()
                
    dfc['split'] = 'train'
    dfc.loc[dfc.index.isin(df_control_test.index), 'split'] = 'test'
    dfc.loc[dfc.index.isin(df_drug_test.index), 'split'] = 'test'
    
    dfc.to_csv(f'/lustre/groups/ml01/projects/CellOT_comparison/cellflow/data/splits/{d}.csv')